# Developing dashboards in Python

<img src="data/images/partners.png" width="75%"/>

Welcome to the technical training on data integration in seasonal forecasting, made by HKV. 

### Table of content

1. [Introduction to Jupyter Notebooks & Application Programming Interfaces (API's)](#section-1)
    - 1.1 Introduction to Jupyter notebook
    - 1.2 Introduction to REST APIs in Python
2. [Working with TAHMO weather station data](#section-2)
3. [Using weahter forecast from Open-meteo](#section-3)
4. [Dashboards development with Solara](#section-4)


## 1. Introduction to Jupyter Notebooks & Application Programming Interfaces (API's) <a name="section-1"></a>

### 1.1 Introduction to Jupyter Notebook
[Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/latest/) are interactive computing environments that allow you to create and share documents containing live code, equations, visualizations, and narrative text. They are widely used in data science, research, and education due to their versatility and ease of use.

Jupyter Notebooks consist of cells which can contain code, text, or visualizations.

#### Running a Cell:
Click on the cell below, and you'll notice a border around it. To run the code within the cell, press **Shift + Enter** or use the "Run" button in the toolbar above.


In [1]:
print('Welcome to the training!')

Welcome to the training!


#### Saving and Closing:
Remember to save your work by clicking the floppy disk icon or using **Ctrl + S**. You can close a notebook when you're done, and the changes will be saved.

In [2]:
# using variables

var = "let's start coding"

print(var)

let's start coding


### 1.2  Introduction to REST APIs in Python
Welcome to the world of REST APIs! If you're curious about how different applications communicate with each other over the web, you're in the right place. In this introduction, we'll explore the basics of REST (Representational State Transfer) APIs and how you can interact with them using Python.

#### What is a REST API?

<img src="data/images/api.png" width="75%"/>

A REST API (Application Programming Interface) is a set of rules and conventions for building and interacting with web services. It allows different software applications to communicate with each other over the internet by using standard HTTP methods, such as GET, POST, PUT, and DELETE.

#### Key Concepts:
1. Resources:
In a REST API, everything is considered a resource. These resources can be data objects, services, or entities, and each is uniquely identified by a URL, often referred to as an endpoint.

2. HTTP Methods:
RESTful APIs use standard HTTP methods to perform different operations on resources:

GET: Retrieve data from a specified resource.
POST: Create a new resource.
PUT: Update an existing resource.
DELETE: Remove a resource.
3. Request and Response:
When you make a request to a REST API endpoint, you send an HTTP request, and the server responds with an HTTP response. The response typically includes the requested data or information about the success or failure of the request.

#### Interacting with REST APIs in Python:
1. Requests Library:
Python provides the requests library, which simplifies the process of sending HTTP requests. You can install it using:

#### Conclusion:
Understanding REST APIs and how to interact with them is a fundamental skill for any programmer. Whether you're building web applications, working with data, or integrating services, the ability to communicate with RESTful APIs using Python opens up a world of possibilities. So, let's dive in and start exploring the power of REST in Python!

# Working with TAHMO weather station data <a name="section-2"></a>

TAHMO is a network of weather stations across Africa. The TAHMO initiative aims to provide high-quality, real-time weather and hydrological data to support various applications, including agriculture, water resource management, and scientific research.

The TAHMO weather stations are strategically placed to gather data on various meteorological parameters such as temperature, humidity, wind speed, and rainfall. The data collected from these stations can be valuable for understanding weather patterns, predicting climate trends, and making informed decisions in sectors like agriculture and water management.

Accessing TAHMO weather station data typically involves using the TAHMO API (Application Programming Interface), which allows developers to retrieve weather data programmatically. See the website below:

In [1]:
from IPython.display import IFrame
IFrame("https://tahmo.org/", '75%',400)

### Example: TAHMO data by API request

In this example we will request data from the TAHMO station using this [API](https://github.com/TAHMO/API-V2-Python-examples/tree/master). This contains a client that makes it easier to use the API. First we need to import the module and create an instance of the API client. Also we need to login using our demo credentials.


In [4]:
# Import the TAHMO module
import TAHMO

# The demo credentials listed below give you access to three pre-defined stations. 
api = TAHMO.apiWrapper()

# set the credentials
api.setCredentials('demo', 'DemoPassword1!')

In the cell below we can list all the TAHMO stations that we have access to. Also we can list al the variables that are recorded by the weather stations.

In [5]:
# list other stations that are available
stations = api.getStations()
print('Account has access to stations: %s' % ', '.join(list(stations)))

API request: services/assets/v2/stations
Account has access to stations: TA00134, TA00252, TA00567


In [6]:
list(stations)

['TA00134', 'TA00252', 'TA00567']

In [7]:
# list available variables

variables = api.getVariables()
print('Available variables in TAHMO API:')
for variable in variables:
    print('%s [%s] with shortcode "%s"' %
          (variables[variable]['description'], variables[variable]['units'], variables[variable]['shortcode']))

API request: services/assets/v2/variables
Available variables in TAHMO API:
Atmospheric pressure [kPa] with shortcode "ap"
Depth of water [mm] with shortcode "dw"
Electrical conductivity of precipitation [mS/cm] with shortcode "ec"
Electrical conductivity of water [mS/cm] with shortcode "ew"
Lightning distance [km] with shortcode "ld"
Lightning events [-] with shortcode "le"
Shortwave radiation [W/m2] with shortcode "ra"
Soil moisture content [m3/m3] with shortcode "sm"
Soil temperature [degrees Celsius] with shortcode "st"
Surface air temperature [degrees Celsius] with shortcode "te"
Vapor pressure [kPa] with shortcode "vp"
Wind gusts [m/s] with shortcode "wg"
Wind speed [m/s] with shortcode "ws"
Temperature of humidity sensor [degrees Celsius] with shortcode "ht"
X-axis level [degrees] with shortcode "tx"
Y-axis level [degrees] with shortcode "ty"
Logger battery percentage [-] with shortcode "lb"
Logger reference pressure [kPa] with shortcode "lp"
Logger temperature [degrees Celsius]

Let's take one of the stations and retrieve the name of the weather stations and the geographical coordinates of the station:

In [8]:
# choose a station
station = 'TA00567'

# get the data
station_data = api.getStations()[station]

print()
print( f"Station name =  {station_data['location']['name']}")
print( f"Longitude =  {station_data['location']['longitude']:.02f}")
print( f"Latitude =  {station_data['location']['latitude']:.02f}")

API request: services/assets/v2/stations

Station name =  Accra Girls SHS
Longitude =  -0.19
Latitude =  5.60


### Retrieve data

In [9]:
# Example 3: Retrieve a pandas dataframe containing the time serie of surface air observations and save to CSV file.

startDate = '2023-01-01'
endDate = '2023-11-22'
variables = ['pr']

df = api.getMeasurements(station, startDate=startDate, endDate=endDate, variables=variables)
df.index.name = 'Timestamp'
df.to_csv('data/timeseries.csv', na_rep='', date_format='%Y-%m-%d %H:%M')
print('Timeseries saved to file "timeseries.csv"')


API request: services/measurements/v2/stations/TA00567/measurements/controlled


Timeseries saved to file "timeseries.csv"


In [10]:
df

,pr
Timestamp,
2023-01-01 00:00:00+00:00,0.0
2023-01-01 00:05:00+00:00,0.0
2023-01-01 00:10:00+00:00,0.0
2023-01-01 00:15:00+00:00,0.0
2023-01-01 00:20:00+00:00,0.0
...,...
2023-11-21 23:40:00+00:00,0.0
2023-11-21 23:45:00+00:00,0.0
2023-11-21 23:50:00+00:00,0.0


In [11]:
import pandas as pd

def load_precip_data():
    df = pd.read_csv("data/timeseries.csv", parse_dates=True, index_col=0).reset_index().rename(columns={"Timestamp" : "date", "pr": "precipitation"})
    df.loc[:,'date'] = df['date'].dt.date
    df = df.groupby('date').max().reset_index().dropna()
    return df

df = load_precip_data()
df

,date,precipitation
0,2023-01-01,0.0
1,2023-01-02,0.0
2,2023-01-03,0.0
3,2023-01-04,0.0
4,2023-01-05,0.0
...,...,...
321,2023-11-18,0.0
322,2023-11-19,0.0
323,2023-11-20,0.0
324,2023-11-21,0.0


#### Interactive chart in Vega-Altair


[Vega-Altair](https://altair-viz.github.io/)  is a handy Python library that allows you to create statistical visualizations using declarative principles without complex programming code. It provides a straightforward way to quickly generate various types of charts.

Altair is built on  [Vega-Lite](https://vega.github.io/vega-lite/), which is a Grammar of Interactive Graphics. Vega-Altair provides a user-friendly way to use this through Python, storing the graphical specifications in JSON (JavaScript Object Notation) format. You can directly view these specifications in any web browser, and coding is easy in JupyterLab, Jupyter Notebook, Microsoft VS-Code, and Google Colab.

In [12]:
import altair as alt

chart =  alt.Chart(df).mark_bar().encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).properties(width=1200, height=200).interactive()
chart

alt.Chart(...)

### Section 2: Open-meteo data

[Open-Meteo](https://github.com/open-meteo/open-meteo) is an open-source weather API and offers free access for non-commercial use. It includes hourly forecasts up to 16 days, but also historic weather. See the website below for more details:


In [2]:
from IPython.display import IFrame
IFrame("https://open-meteo.com/", "75%", 400)

In [14]:
import requests


def get_ecmwf_precipitation(api_key, lon, lat):
    base_url = "https://api.open-meteo.com/v1/forecast"
    
    # Specify the parameters for the ECMWF precipitation forecast
    params = {
        "longitude" : lon,
        "latitude" : lat,
        "daily" : "precipitation_sum",
        # "start_date" : startDate,
        # "end_date" : endDate,
        "past_days" : 90,
        "timezone" : "auto",
        "hourly" : "precipitation",
        "start" : "current",
        "forecast_days" : 10,
        "models" : "ecmwf_ifs04", 
        "apikey" : api_key,
    }

    try:
        # Make a request to the Open-Meteo API
        response = requests.get(base_url, params=params)
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"Error: {e}")

data = get_ecmwf_precipitation('', lon=station_data['location']['longitude'], lat=station_data['location']['latitude'])

In [15]:
df = pd.DataFrame.from_dict(data['hourly'])

df['time'] = pd.to_datetime(df['time'])
df.loc[:,'date'] = df['time'].dt.date
df['date'] = pd.to_datetime(df['date'])
df = df[['date', 'precipitation']].dropna()
df = df.groupby('date').max().reset_index()# .set_index('date')

df

,date,precipitation
0,2023-08-27,0.1
1,2023-08-28,1.0
2,2023-08-29,0.4
3,2023-08-30,3.5
4,2023-08-31,0.5
...,...,...
95,2023-11-30,0.1
96,2023-12-01,0.3
97,2023-12-02,0.1
98,2023-12-03,0.4


In [16]:
chart =  alt.Chart(df).mark_bar().encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).properties(width=1200, height=300).interactive().configure_mark(color='orange')
chart

alt.Chart(...)

## 4. Introduction to web apps in Solara 

[Solara](https://solara.dev/) is a python library for data-focused web apps which you can run in a Jupyter notebook as well as in production-grade web frameworks (FastAPI, Starlette, Flask, ...). It uses IPywidgets for UI components which saves you from having to learn Javascript and CSS. 

In [17]:
station_list = ["TA00134", "TA00252", "TA00567"]
station_data = {}

for station in station_list:
    station_data[station] = api.getStations()[station]

API request: services/assets/v2/stations
API request: services/assets/v2/stations
API request: services/assets/v2/stations


In [18]:
import solara
import ipyleaflet
import TAHMO
from ipywidgets import HTML

# Create a TAHMO API wrapper and set credentials
api = TAHMO.apiWrapper()
api.setCredentials('demo', 'DemoPassword1!')

# Define reactive variables for station data
station = solara.reactive('TA00134')

def set_station(value):
    station.value = value

@ solara.component
def StationSelect():
    """Solara component for a station selection dropdown."""
    solara.Select(label="station", values=station_list, value=station.value, on_value=set_station)
    
@solara.component
def View():
    """Solara component for displaying a map view with a marker for the selected station."""
    m = ipyleaflet.Map(center=[station_data[station.value]['location']['latitude'], 
                               station_data[station.value]['location']['longitude']], 
                       zoom=9, 
                        scroll_wheel_zoom=True, )
    marker = ipyleaflet.Marker(location=(station_data[station.value]['location']['latitude'], 
                               station_data[station.value]['location']['longitude']), draggable=False)
    marker.popup = HTML(station_data[station.value]['location']['name'])
    m.add_layer(marker)
    solara.display(m)

@solara.component
def Page():
    """Solara component for a page with two cards: View and StationSelect."""
    
    with solara.Card():
        View()

    with solara.Card():
        StationSelect()


Page()

Cannot show ipywidgets in text

In [27]:
# Define reactive variables for station data
station = solara.reactive('TA00252')

def set_station(value):
    station.value = value
    
def request_precip_data(station, variables=['pr'], startDate='2023-01-01', endDate='2023-11-22'):
    """Request precipitation data from the TAHMO API and return a pandas dataframe."""
    df = api.getMeasurements(station, startDate=startDate, endDate=endDate, variables=variables)
    if df.empty:
        df = pd.DataFrame(columns=['date', 'precipitation'])
        return df
    else:
        df.index.name = 'Timestamp'
        df = df.reset_index()
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.loc[:,'date'] = df['Timestamp'].dt.date
        df = df.drop(columns=['Timestamp']).groupby('date').max().reset_index().dropna()
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename(columns={"pr": "precipitation"})
        return df

@ solara.component
def StationSelect():
    """Solara component for a station selection dropdown."""
    solara.Select(label="station", values=station_list, value=station.value, on_value=set_station)

@solara.component
def Timeseries():
    """Solara component for a timeseries chart of precipitation."""	
    df = request_precip_data(station.value)
    chart =  alt.Chart(df).mark_bar().encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).properties(width=950, height=350).interactive()
    solara.display(chart)

@solara.component
def Page():
    """Solara component for a page with two cards: View and StationSelect."""
    with solara.Card():
        Timeseries()
    with solara.Card():
        StationSelect()

Page()

API request: services/measurements/v2/stations/TA00252/measurements/controlled


Cannot show ipywidgets in text

In [21]:
station.value

'TA00252'